In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline 

from skimage import io, color, filters, feature, restoration
from skimage.transform import resize, rotate
from skimage.color import rgb2gray
from scipy.spatial.distance import squareform
from scipy.misc import imread
import pandas as pd
import matplotlib.cm as cm
from scipy import ndimage, misc
import pickle

import sys
sys.path.append("..")

resized_image = io.imread('../data/medium/104_img.png')

subset = [174, 96, 260, 116, 294, 180, 301, 36, 256, 189, 37, 175, 165, 446, 87, 203, 271, 181, 25, 302, 410, 444, 183, 193, 0, 24, 254, 127, 339]

sim_image_list = ['../data/medium/{}_img.png'.format(i) for i in subset]

sim_image_list

io.imshow(resized_image)

sys.path.append("..")
from src.standardizer import standardizer
from src.image_generator import imageGenerator
from src_test.imageData_generator import ImageGenerator
from src_test.predict_images import LineScrubber


ruled = glob.glob('../Data/Medium/*')



one_image = '../data/medium/104_img.png'

image_prep = standardizer(sim_image_list, one_image)
image_prep.greyscale(29)
image_prep.standardize()
image_prep.binarize(0.85)

# fig, ax = plt.subplots(8, 10, figsize = (20, 10))
for i in range(29):
    fig, ax = plt.subplots()
    ax.imshow(image_prep.binarized_images[i], cmap='gray')
    print(image_prep.image_list[i])
    plt.show()
# plt.show()

image_prep.visualize(7, 4)

io.imshow(image_prep.binarized_images[19])

image_prep.image_list[19].split('/')[3].split('.')[0]

img_name = image_prep.image_list[19].split('/')[3].split('.')[0]


bin_image = image_prep.binarized_images[19]
grey_image = image_prep.greyscale_image_list[19]

images = ImageGenerator(bin_image, grey_image, img_name)
images.pad(15)

# io.imread('../data/gray/all/lines/36_img_200_1.png').shape

gen_list = [88, 118, 144]

# for g in gen_list:
# images.gen_data(205)



gray = images.gray_padded_image
binar = images.bin_padded_image


io.imshow(gray, cmap='gray')

What if we only visit pixels where pixel intensity is < 216? It would be quicker/more efficient

np.argwhere(gray <= 235)

gray[29, 138]

gray.shape
gray.shape[0]*gray.shape[1]

np.argwhere(gray <= 216.0).shape

Now, were visiting 15,187 pixels instead of 144,910

visit_list = np.argwhere(gray <= 225.0)

visit_list.shape

for x in visit_list:
    print(x[0], x[1])
    plot_frame(gray, x[0]-15, x[1]-15, 30)
    plt.show()
    window = gray[x[0]-15:x[0]-15+30, x[1]-15:x[1]-15+30]
    pixel_value = window[15, 15]
    print(pixel_value)

io.imshow(binar)

model_path = '../models/models/barebones.sav'

from src_test import modeling
sys.modules['modeling'] = modeling

pickle.load(open(model_path, 'rb'))

scrubber = LineScrubber(binar, gray, '../models/models/barebones.sav', 'test')

scrubber.scrub()

def plot_frame(zoom, row_index, col_index, size):
    masked_window = np.random.random((zoom.shape[0],zoom.shape[1]))
    masked_window[row_index:row_index+size, col_index:col_index+size] = 1
    masked_window = np.ma.masked_where(masked_window != 1, masked_window)

    masked_pixel = np.random.random((zoom[row_index:row_index+size, col_index:col_index+size].shape[0],zoom[row_index:row_index+size, col_index:col_index+size].shape[1]))
    masked_pixel[15,15] = 1
    masked_pixel = np.ma.masked_where(masked_pixel != 1, masked_pixel)

    masked_pixel1 = np.random.random((zoom[row_index:row_index+size, col_index:col_index+size].shape[0],zoom[row_index:row_index+size, col_index:col_index+size].shape[1]))
    masked_pixel1[8:23, 15] = 1
    masked_pixel1 = np.ma.masked_where(masked_pixel1 != 1, masked_pixel1)
    
    window = zoom[row_index:row_index+size, col_index:col_index+size]
    colored_percentage = np.count_nonzero(window==0)/(30**2)
    pixel_value = window[15, 15]
    window_sobel = ndimage.sobel(window, axis=0)
    above_area = np.mean(window_sobel[8:16, 15])
    below_area = np.mean(window_sobel[16:23, 15])
    sobel_value = window_sobel[15, 15]
    
    # Overlay the two images
    fig, ax = plt.subplots(1, 3)
    ax.ravel()
    ax[0].imshow(zoom, cmap='gray')
    ax[0].imshow(masked_window, cmap='prism', interpolation='none')
    # ax[0].imshow(masked_pixel, cmap=cm.jet, interpolation='none')
    ax[0].set_title('Colored: {}'.format(round(colored_percentage, 2)))
    ax[1].imshow(window, cmap='gray')
    ax[1].imshow(masked_pixel, cmap='prism', interpolation='none')
    ax[1].set_title('Pixel Value: {}'.format(pixel_value))
    ax[2].imshow(window_sobel)
    ax[2].imshow(masked_pixel1, cmap='jet')
    ax[2].imshow(masked_pixel, cmap='prism', interpolation='none')
#     return above_area, below_area, colored_percentage


def get_label():
    label = input()
    if label == 'l':
        label_for_entry = 1
    elif label == '':
        label_for_entry = 1
    else:
        label_for_entry = 0
    return label_for_entry



def update_dict(my_dict, gray_pixel_value, mean_pixel_value, colored_percentage, sobel_gradient, label):
    my_dict['gray_pixel_value'].append(gray_pixel_value)
    my_dict['mean_gray_pixel_value'].append(mean_pixel_value)
    my_dict['bin_percentage_colored'].append(colored_percentage)
    my_dict['sobel_gradient'].append(sobel_gradient)
    my_dict['label'].append(label)
#     return my_dict

def add_label_history(df):
    df['label_t1'] = ""
    df['label_t2'] = ""
    df['label_t3'] = ""
    for idx, row in df.iterrows():
        if idx > 0:
            df.at[idx,'label_t1'] = list(df[df.index == idx - 1].label)[0]
            if idx > 1:
                df.at[idx,'label_t2'] = list(df[df.index == idx - 2].label)[0]
                if idx > 2:
                    df.at[idx,'label_t3'] = list(df[df.index == idx - 3].label)[0]

def gen_data(row):  
    my_dict = {'sobel_above': [], 'sobel_below': [], 'percentage_colored': [], 'label': []}
    for i in range(1, 900):
        size = 30
        row_index = row
        col_index = i
        plot_frame(zoom, row_index, col_index, size)
        plt.show()
        window = zoom[row_index:row_index+size, col_index:col_index+size]
        colored_percentage = np.count_nonzero(window==0)/(30**2)
        above_area = np.mean(window_sobel[8:16, 15])
        below_area = np.mean(window_sobel[16:23, 15])
        pixel_value = window[15, 15]
        if pixel_value == 0.0:
            label = get_label()
            update_dict(my_dict, above_area, below_area, colored_percentage, label)
    df = pd.DataFrame.from_dict(my_dict)
    add_label_history(df)
    return df

Image Classification:
- If Binary image pixel value is 0.0:
    - FIRST: Label frame as a line or drawing (this label will be determined by looking at the binary frame, its easier to determine)
    - save binary frame to binary directory/line or /drawing
    - save grayscale frame to grayscale directory/line or /drawing
    - place grayscale features in features dictionary 

my_dict = {'gray_pixel_value': [], 'mean_gray_pixel_value': [], 'bin_percentage_colored': [], 'sobel_gradient': [], 'label': []}

size = 30
row_idx = 205

col_idx = 0
plot_frame(gray, row_idx, col_idx, size)
plot_frame(binar, row_idx, col_idx, size)
plt.show()
gray_window = gray[row_idx:row_idx+size, col_idx:col_idx+size]
bin_window = binar[row_idx:row_idx+size, col_idx:col_idx+size]
gray_pixel_value = gray_window[15, 15]
bin_pixel_value = bin_window[15, 15]
mean_pixel_value = np.mean(gray_window)
gray_window_sobel = ndimage.sobel(gray_window, axis=0)
colored_percentage = np.count_nonzero(bin_window==0)/(30**2)
above_area = np.mean(gray_window_sobel[8:16, 15])
below_area = np.mean(gray_window_sobel[16:23, 15])
sobel_gradient = above_area - -below_area
if bin_pixel_value == 0.0:
    label = get_label()
    update_dict(my_dict, gray_pixel_value, mean_pixel_value, colored_percentage, sobel_gradient, label)
            
            
print(my_dict)
io.imshow(bin_window, cmap='gray')
    
# window = zoom[row_index:row_index+size, col_index:col_index+size]
# colored_percentage = np.count_nonzero(window==0)/(30**2)
# above_area = np.mean(window_sobel[8:16, 15])
# below_area = np.mean(window_sobel[16:23, 15])
# label = get_label()
# update_dict(my_dict, above_area, below_area, colored_percentage, label)





















What is the average pixel value of "white space"? Its not 255

whitespace = []
for i in range(400):
    size = 30
    row_index = 10
    col_index = i
#     plot_frame(result, row_index, col_index, size)
    plot_frame(result_g, row_index, col_index, size)
    plt.show()
    window = result_g[row_index:row_index+size, col_index:col_index+size]
    pixel_value = window[15, 15]
    whitespace.append(pixel_value)
    


np.mean(whitespace)



df_227

df_227.to_csv



























